<a href="https://colab.research.google.com/github/leehon94/Automated-Product-Identification_PJS/blob/master/keras_Xception_PJS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras import optimizers
from keras import layers
from keras import models

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


base_dir = r'/content/gdrive/My Drive/MVP Datensatz/'
train_dir = r'/content/gdrive/My Drive/MVP Datensatz/Trainingsdatensatz5'


In [0]:


#Beispiel für mögliche Data Augmentations
train_datagen = ImageDataGenerator(featurewise_center=False,
                          samplewise_center=False,
                          featurewise_std_normalization=False,
                          samplewise_std_normalization=False,
                          zca_whitening=False,
                          zca_epsilon=1e-06,
                          rotation_range=3,
                          width_shift_range=[-1, 0, +1],
                          height_shift_range=[-1, 0, +1],
                          brightness_range=None, 
                          shear_range=0.1, 
                          zoom_range=0.2,
                          channel_shift_range=0.0, 
                          fill_mode='nearest', 
                          cval=0.0,
                          horizontal_flip=False, 
                          vertical_flip=False, 
                          rescale=1./255, 
                          preprocessing_function=None, 
                          data_format='channels_last', 
                          validation_split=0.2, 
                          interpolation_order=1, 
                          dtype='float32')


valid_datagen = ImageDataGenerator(
                          rescale=1./255,
                          validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    subset='training')

validation_generator = valid_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    subset='validation')



In [0]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

from keras.applications import Xception
base_model = Xception(weights='imagenet',
  include_top=False,
  input_shape=(150,150,3))

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
batch_size = 10
                      

In [9]:
      

def extract_features(directory, sample_count):
  features = np.zeros(shape=(sample_count, 4, 4, 512)) 
  labels = np.zeros(shape=(sample_count))
  generator = datagen.flow_from_directory(
    directory,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical')
  i=0
  for inputs_batch, labels_batch in generator:
    features_batch = base_model.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
      break
    return features, labels
train_features, train_labels = extract_features(train_dir, 100)
#validation_features, validation_labels = extract_features(validation_dir, 303)




IndentationError: ignored

In [0]:

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer for 5 classes
predictions = Dense(5, activation='softmax')(x)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(train_generator,  
  steps_per_epoch=139,
  epochs=11,
  validation_data = validation_generator)